In [32]:
import pandas as pd

In [80]:
df = pd.read_csv("Dataset\dataset_siap.csv")
# print(df)

In [81]:
df = df.astype(str).apply(lambda x: x.str.lower())
print(df)

                                               Kalimat Kelas
0         tampilkan kartu hasil studi untuk semester 1   id 
1         tampilkan kartu hasil studi untuk semester 2   id 
2         tampilkan kartu hasil studi untuk semester 3   id 
3         tampilkan kartu hasil studi untuk semester 4   id 
4         tampilkan kartu hasil studi untuk semester 5   id 
..                                                 ...   ...
325  apa yang menjadi alasan/landasan anda dalam me...   ood
326           seberapa penting bisa berbahasa inggris?   ood
327     kapan tere liye menerbitkan buku pertama kali?   ood
328                         mengapa kita perlu minum ?   ood
329         siapa penulis puisi pada suatu hari nanti?   ood

[330 rows x 2 columns]


In [82]:
import re

In [83]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text


In [87]:
# Buat kolom tambahan untuk data description yang telah diremovepunctuation   
df['Kalimat']= df['Kalimat'].apply(clean_punct)

In [88]:
print(df)

                                               Kalimat Kelas
0         tampilkan kartu hasil studi untuk semester 1   id 
1         tampilkan kartu hasil studi untuk semester 2   id 
2         tampilkan kartu hasil studi untuk semester 3   id 
3         tampilkan kartu hasil studi untuk semester 4   id 
4         tampilkan kartu hasil studi untuk semester 5   id 
..                                                 ...   ...
325  apa yang menjadi alasanlandasan anda dalam mem...   ood
326           seberapa penting bisa berbahasa inggris    ood
327     kapan tere liye menerbitkan buku pertama kali    ood
328                         mengapa kita perlu minum     ood
329         siapa penulis puisi pada suatu hari nanti    ood

[330 rows x 2 columns]


In [73]:
import nltk

In [90]:
df['Kalimat'] = df.apply(lambda row: nltk.word_tokenize(row['Kalimat']), axis=1)
print(df)

                                               Kalimat Kelas
0    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
1    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
2    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
3    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
4    [tampilkan, kartu, hasil, studi, untuk, semest...   id 
..                                                 ...   ...
325  [apa, yang, menjadi, alasanlandasan, anda, dal...   ood
326      [seberapa, penting, bisa, berbahasa, inggris]   ood
327  [kapan, tere, liye, menerbitkan, buku, pertama...   ood
328                      [mengapa, kita, perlu, minum]   ood
329  [siapa, penulis, puisi, pada, suatu, hari, nanti]   ood

[330 rows x 2 columns]
